In [ ]:
!pip install datasets bitsandbytes accelerate sentencepiece git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git # transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-jvxnr4fe
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-jvxnr4fe
  Resolved https://github.com/huggingface/peft.git to commit 2822398fbe896f25d4dac5e468624dc5fd65a51b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-37or_6zk
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-37or_6zk
  Resolved https://github.com/huggingface/transformers.git to commit d04ec99bec8a0b432fc03ed60cea9a1a20ebaf3c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... 

In [3]:
import torch
from peft import PeftModel
import transformers

# from transformers import LLamaTokenizer, LLamaForCausalLM, GenerationConfig
from transformers import LLaMAForCausalLM, LLaMATokenizer, GenerationConfig


In [4]:
assert torch.cuda.is_available()

In [6]:
!ls -lah

total 87M
drwxr-xr-x 14 root root 4.0K Apr 28 19:59  .
drwx------  1 root root 4.0K Apr 28 19:58  ..
-rw-r--r--  1 root root   42 Apr 28 12:29  .dockerignore
drwxr-xr-x  8 root root  163 Apr 28 12:29  .git
drwxr-xr-x  3 root root   23 Apr 28 12:29  .github
-rw-r--r--  1 root root  136 Apr 28 12:29  .gitignore
drwxr-xr-x  2 root root  177 Apr 28 19:59  .ipynb_checkpoints
-rw-r--r--  1 root root  20K Apr 28 12:29  DATA_LICENSE
-rw-r--r--  1 root root  635 Apr 28 12:29  Dockerfile
-rw-r--r--  1 root root  12K Apr 28 12:29  LICENSE
-rw-r--r--  1 root root  17K Apr 28 12:29  README.md
-rw-r--r--  1 root root 502K Apr 28 19:59 '[backup]_batch_size_changes_generate_output.ipynb'
-rw-r--r--  1 root root  22M Apr 28 12:29  alpaca_data.json
-rw-r--r--  1 root root  22M Apr 28 12:29  alpaca_data_cleaned_archive.json
-rw-r--r--  1 root root  42M Apr 28 12:29  alpaca_data_gpt4.json
-rw-r--r--  1 root root 627K Apr 28 19:50  create_t2c_dataset_and_show_max_pad_size.ipynb
-rw-r--r--  1 root root  643

In [8]:
BASE_MODEL = "decapoda-research/llama-7b-hf"
LORA_WEIGHTS = "t2c_concode_220428"#"tloen/alpaca-lora-7b"

model = LLaMAForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    LORA_WEIGHTS,
    torch_dtype=torch.float16,
)

tokenizer = LLaMATokenizer.from_pretrained(BASE_MODEL)
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
tokenizer.padding_side = "left"
# not sure how necessary this part is, not sure if tloen/alpaca-lora-7b was even trained with EOS and BOS tokens
model.config.bos_token_id = 1
model.config.eos_token_id = 2

model.eval()

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LLaMAForCausalLM(
      (model): LLaMAModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
        (layers): ModuleList(
          (0-31): 32 x LLaMADecoderLayer(
            (self_attn): LLaMAAttention(
              (q_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=4096, bias=False)
                )
              )
              (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
              (v_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
          

In [9]:
def generate_prompt(instruction, input):
    return f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"

instruction = 'Combine the question and answer into an image caption as succinctly as possible. Be sure to include the phrase "a photo of". Do not draw false conclusions.'
inputs = ['Is this a baseball game? yes', 'Is this a baseball game? no']
prompts = [generate_prompt(instruction, inp) for inp in inputs]
tokenized_inputs = tokenizer(prompts, padding=True, truncation=True, return_tensors="pt").to('cuda')

generation_config = GenerationConfig(max_new_tokens=128)

with torch.no_grad():
    full_output = model.generate(
        **tokenized_inputs,
        generation_config=generation_config
    )

    # same tokenized input, but we index just into the parts for the second input
    single_output = model.generate(
        **{k: v[1:] for k, v in tokenized_inputs.items()},
        generation_config=generation_config
    )

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [10]:
# different outputs, even though we followed identical tokenization
# `... A photo of people playing a game.` vs `... This is not a baseball game.`
print(tokenizer.batch_decode(full_output, skip_special_tokens=False)[-1])
print('-' * 80)
print(tokenizer.batch_decode(single_output, skip_special_tokens=False)[0])

 Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Combine the question and answer into an image caption as succinctly as possible. Be sure to include the phrase "a photo of". Do not draw false conclusions.

### Input:
Is this a baseball game? no

### Response:
###
### Image:
### photo of a baseball game 
###
### Instruction:
Generate a description of the request that appropriately completes it.

### Input:
A photo of a baseball game
###
### Response:
a photo of a baseball game 
###
--------------------------------------------------------------------------------
 Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Combine the question and answer into an image caption as succinctly as possible. Be sure to include the phrase "a photo of". Do 

In [33]:
import json

def generate_prompt(instruction, input):
    return f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"

def generate_test_prompt(data_point, train = False):
    print(train)
    # sorry about the formatting disaster gotta move fast
    if data_point["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}

### Input:
{data_point["input"]}

### Response:
{data_point["output"] if train else ''}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}

### Response:
{data_point["output"] if train else ''}"""


lst = json.load(open('t2c_concode/t2c_answers.json', 'rb'))
inputs = lst# [lst[0]]
# instruction = 'Combine the question and answer into an image caption as succinctly as possible. Be sure to include the phrase "a photo of". Do not draw false conclusions.'
# inputs = ['Is this a baseball game? yes', 'Is this a baseball game? no']
prompts = [generate_test_prompt(inp) for inp in inputs]
# prompts = [generate_prompt(instruction=inp['instruction'], 
#                            input=inp['input']) 
#            for inp in inputs
#           ]
tokenized_inputs = tokenizer(prompts, padding=True, truncation=True, return_tensors="pt").to('cuda')

generation_config = GenerationConfig(max_new_tokens=128)

with torch.no_grad():
    full_output = model.generate(
        **tokenized_inputs,
        generation_config=generation_config
    )

print(tokenizer.batch_decode(full_output, skip_special_tokens=False)[-1])

#     # same tokenized input, but we index just into the parts for the second input
#     single_output = model.generate(
#         **{k: v[1:] for k, v in tokenized_inputs.items()},
#         generation_config=generation_config
#     )

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.19 GiB (GPU 0; 23.70 GiB total capacity; 13.80 GiB already allocated; 2.95 GiB free; 19.69 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [31]:
prompts[0]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n### Instruction:\nGenerate java code\nactually walks the bag to make sure the count is correct and resets the running total \n\n### Input:\n Object _current \n int _total \n DefaultMapBag _parent \n Map _map \n int _mods \n Iterator _support \n\n boolean add \n boolean add \n Object next \n boolean containsAll \n boolean containsAll \n void clear \n boolean isEmpty \n boolean hasNext \n void remove \n boolean remove \n boolean remove \n Map getMap \n int modCount \n boolean contains \n Iterator iterator \n boolean removeAll \n int size \n boolean addAll \n int hashCode \n boolean equals \n Object[] toArray \n Object[] toArray \n Set uniqueSet \n void setMap \n String toString \n int getCount \n List extractList \n boolean retainAll \n boolean retainAll\n\n### Response:\n'

In [23]:
full_output = model.generate(
    **tokenized_inputs,
    generation_config=generation_config
)

In [17]:
    full_output = model.generate(
        **tokenized_inputs,
        generation_config=generation_config
    )

 Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate java code
actually walks the bag to make sure the count is correct and resets the running total 

### Input:
 Object _current 
 int _total 
 DefaultMapBag _parent 
 Map _map 
 int _mods 
 Iterator _support 

 boolean add 
 boolean add 
 Object next 
 boolean containsAll 
 boolean containsAll 
 void clear 
 boolean isEmpty 
 boolean hasNext 
 void remove 
 boolean remove 
 boolean remove 
 Map getMap 
 int modCount 
 boolean contains 
 Iterator iterator 
 boolean removeAll 
 int size 
 boolean addAll 
 int hashCode 
 boolean equals 
 Object[] toArray 
 Object[] toArray 
 Set uniqueSet 
 void setMap 
 String toString 
 int getCount 
 List extractList 
 boolean retainAll 
 boolean retainAll

### Response:
 void walk 
###



In [24]:
print(tokenizer.batch_decode(full_output, skip_special_tokens=False)[-1])


 Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Generate java code
actually walks the bag to make sure the count is correct and resets the running total 

### Response:
int function ( ) { _total = extractList ( ). size ( ) ; return _total ; }


In [25]:
prompts

['Below is an instruction that describes a task. Write a response that appropriately completes the request.\n### Instruction:\nGenerate java code\nactually walks the bag to make sure the count is correct and resets the running total \n\n### Response:\nint function ( ) { _total = extractList ( ) . size ( ) ; return _total ; }']